In [ ]:
import os
import sys

import evo_helpers as hlp
import evo_constants as CTS

In [ ]:
hlp.welcome()

In [ ]:
from importlib import reload
reload(evo_helpers)

opt = hlp.option_menu()

In [ ]:
## obtener las bases de datos del usuario

from importlib import reload
reload(evo_helpers)

if opt == 1:
    evomining = EvoMining(cts.EXML_DB["central"],
                          cts.EXMPL_DB["genomes"],
                          cts.EXMPL_DB["nat_prods"])
    evomining.run()
else: 
    user_dbs = hlp.get_dbs()


In [ ]:
#from importlib import reload
#reload(evo_helpers)

genomes_db = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17"
Rast_ids = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17Rast.ids"
hlp.make_all_evo_headers(genomes_db, Rast_ids)